In [33]:
# Read credentials from a file
import json
credentials = json.loads(open("credentials.json",'r').read())

In [111]:
# Open browser to engine the application
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--mute-audio")

driver = webdriver.Chrome(options=chrome_options)

In [72]:
# Adjust screen size
driver.set_window_size(1920, 1080)

In [76]:
# Printscreen definition

import time
import os

def printScreen(nameOfFile = str(time.time())):
    date = time.strftime("%d-%m")
    screenshot_dir = "./screenshot/" + date + "/"
    if not os.path.exists(screenshot_dir):
        os.makedirs(screenshot_dir)
    nameOfFile = screenshot_dir + nameOfFile + ".png"
    driver.save_screenshot(nameOfFile)

In [74]:
# -- Telemetry -- Print Screen
# printScreen(str(time.time()))
printScreen("Screenshot-taken")

In [112]:
# Login
driver.delete_all_cookies() # Idenpotent garantee
driver.get('https://play.pokemonshowdown.com')
time.sleep(1.5)
login_button = driver.find_element(By.NAME, 'login')
login_button.click()
driver.find_element(By.NAME,"username").send_keys(credentials['user'])
driver.find_element(By.CSS_SELECTOR,'[type=submit]').click()
time.sleep(0.5)
driver.find_element(By.NAME,"password").send_keys(credentials['password'])
driver.find_element(By.CSS_SELECTOR,'[type=submit]').click()
time.sleep(1)
printScreen()

In [43]:
# Accept the challenge
name = "acceptChallenge"
time.sleep(0.5)
button = driver.find_element(By.NAME, name)
if not button:
    print("No challenge available!")
    raise Exception("No challenge available!")
print(button)
button.click()
time.sleep(1.4)
printScreen()

<selenium.webdriver.remote.webelement.WebElement (session="6772382333d83c58f964023c5e7d5889", element="f.944F1CCC6291189293446BE8DF33ADEA.d.C370CE5C5FF4F5463DAA8E11B5627BE5.e.64")>


### Functions to Read Battle Data
The functions are:
- `getAdversaryPokes`: Get the adversary's pokemons available(not hidden) in the battle.
- `getMoves`: Get the available moves for your pokemon active in the battle.
- `ChooseMove`: Choose a move to use in the battle.

In [127]:
pokemonData = json.loads(open("./data/dex.json",'r', encoding='utf8').read())
def getPokeDataByName(name: str):
    pokeFound = pokemonData.get(name)
    if not pokeFound:
        return None
    print(pokeFound.get('types'))
    type1 = str(pokeFound['types'][0]),
    type2 = str(pokeFound['types'][1]) if len(pokeFound['types']) > 1 else None,
    return {
        'name': name,
        'type1': type1,
        'type2': type2
    }

In [119]:
# Get the adversary pokes
from selenium.webdriver.common.action_chains import ActionChains
def getAdversaryPokes():
    enemyPokesSelector = '.trainer-far .teamicons .picon'
    enemyPokes = driver.find_elements(By.CSS_SELECTOR, enemyPokesSelector)
    pokeDataToReturn = []
    for poke in list(enemyPokes):
        ActionChains(driver).move_to_element(poke).perform()
        time.sleep(0.5)
        innerHTML = driver.find_elements(By.CSS_SELECTOR, '#tooltipwrapper')[0].get_attribute('innerHTML')
        if ("HP" not in innerHTML): continue
        pokeHP = driver.find_elements(By.CSS_SELECTOR, '#tooltipwrapper')[0].get_attribute('innerHTML').split('HP:</small> ')[1].split("%")[0]
        pokeName = poke.get_attribute("aria-label").split("(")[0].strip() if poke.get_attribute("aria-label").__contains__("(") else poke.get_attribute("aria-label").strip()
        if (pokeName == "Not revealed"): continue
        pokeFound = getPokeDataByName(pokeName.replace(" ","").lower())
        if (not pokeFound): 
            print("Pokemon not found: " + pokeName)
            continue
        barName = driver.find_element(By.CSS_SELECTOR, '[aria-label="Active Pokemon"] .statbar.lstatbar').get_attribute('innerHTML').split('strong>')[1].split('<')[0]
        # print(barName, pokeName)
        data = {
            'name': pokeName,
            'HP': int(pokeHP) if 'fainted' not in pokeHP else 0,
            'isActive': pokeName.strip() == barName.strip(),
            'type1': pokeFound['type1'],
            'type2': pokeFound['type2'],
        }
        pokeDataToReturn.append(data)
    return pokeDataToReturn


In [106]:
# Get the available moves
moveData = json.loads(open("./data/moves.json",'r').read())
def separateMove(move: str):
    # Tera Blast Normal 16/16
    # And convert to type, name, pp, allPP -> Normal, Tera Blast, 16, 16
    splited = move.split(' ')
    ppStr = splited[-1]
    pp = ppStr.split('/')[0]
    allPP = ppStr.split('/')[1]
    name = " ".join(splited[0:len(splited)-2])
    type = splited[len(splited)-2]
    return type, name, pp, allPP
def getMoves():
    movesDataToReturn = []
    moves = driver.find_elements(By.CSS_SELECTOR, '[name="chooseMove"]')
    index = 1
    for move in list(moves):
        normalized = move.get_attribute('innerText').replace('\n', ' ').replace('  ', ' ').strip()
        move = normalized
        # data = move.replace('\n', ' ').replace('  ', ' ').strip().split()
        # moveType = data[len(data)-2]
        # moveName = " ".join(data[0:len(data)-2])
        # movePP = data[-1]
        moveType, moveName, movePP, allPP = separateMove(normalized)
        moveFound = moveData[moveName.replace(" ","").lower()] if moveName.replace(" ","").lower() in moveData else {}
        data = {
            'type': moveType,
            'name': moveName,
            'pp': movePP,
            'allPP': allPP,
            # 'pp': movePP.split('/')[0],
            # 'allPP': movePP.split('/')[1],
            'power': moveFound['basePower'] if 'basePower' in moveFound else 0,
            'accuracy': moveFound['accuracy'] if 'accuracy' in moveFound else 100,
            'index': index
        }
        index += 1
        movesDataToReturn.append(data)
    return movesDataToReturn

In [ ]:
# Choose random move
import random
def chooseRandomMove():
    moves = driver.find_elements(By.CSS_SELECTOR, '[name="chooseMove"]')
    if len(moves) == 0:
        print("No moves available!")
        return
        # raise Exception("No moves available!")
    move = moves[random.randint(0, len(moves)-1)]
    move.click()
    time.sleep(0.5)

In [120]:
# Get available switches
def getSwitches():
    switchesDataToReturn = []
    switches = driver.find_elements(By.CSS_SELECTOR, '[name="chooseSwitch"]')
    index = 1
    for switch in list(switches):
        switchName = switch.get_attribute('innerText').replace('\n', ' ').replace('  ', ' ').strip()
        pokeFound = getPokeDataByName(switchName.replace(" ","").lower())
        if (not pokeFound): 
            print("Pokemon not found: " + switchName)
            switchesDataToReturn.append({ 'name': switchName, 'index': index })
            index += 1
            continue
        data = {
            'name': switchName,
            'index': index,
            'type1': pokeFound['type1'],
            'type2': pokeFound['type2'],
        }
        index += 1
        switchesDataToReturn.append(data)
    return switchesDataToReturn

In [113]:
# Choose random switch
def chooseRandomSwitch():
    switches = driver.find_elements(By.CSS_SELECTOR, '[name="chooseSwitch"]')
    if len(switches) == 0:
        print("No switches available!")
        return
        # raise Exception("No switches available!")
    switch = switches[random.randint(0, len(switches)-1)]
    switch.click()
    time.sleep(0.5)

In [141]:
# Check if can do a move
def canDoAMove():
    return driver.find_elements(By.CSS_SELECTOR, '[name="chooseMove"]').__len__() > 0


In [142]:
# Check if can do a switch
def canBeSwitch():
    return driver.find_elements(By.CSS_SELECTOR, '[name="chooseSwitch"]').__len__() > 0

In [146]:
# Await for the adversary play
def awaitAdversaryPlay():
    while not canDoAMove() and not canBeSwitch():
        return
    print('Waiting...')
    time.sleep(1.5)

In [147]:
# RandomPlay
def randomPlay():
    while True:
        if not canDoAMove() and not canBeSwitch():
            awaitAdversaryPlay()
        elif canDoAMove() and not canBeSwitch():
            chooseRandomMove()
            break
        elif not canDoAMove() and canBeSwitch():
            chooseRandomSwitch()
            break
        else:
            if random.randint(0, 1) == 0:
                chooseRandomMove()
            else:
                chooseRandomSwitch()
            break

In [150]:
# moves = getMoves()
# enemyPokes = getAdversaryPokes()
chooseRandomSwitch()


# Simulate a battle
randomPlay()

Waiting...
Waiting...
Waiting...
Waiting...


KeyboardInterrupt: 